In [34]:
!pip install chess

^C
ERROR: Operation cancelled by user


In [35]:
import chess
import pandas as pd



In [36]:
import numpy as np
import chess.engine
import chess.pgn


In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split


In [38]:
import dask as dd

In [39]:

# import csv

# def pgn_to_fen_csv(pgn_file, output_csv):
#     with open(pgn_file) as pgn, open(output_csv, 'w', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         # Write the header row
#         writer.writerow(["White", "Black", "Result", "Move_Number", "Move", "FEN"])
        
#         game_count = 0
        
#         while True:
#             game = chess.pgn.read_game(pgn)
#             if game is None:
#                 break
            
#             game_count += 1
#             board = game.board()
#             moves = list(game.mainline_moves())
            
#             # Extract game metadata
#             white = game.headers.get("White", "Unknown")
#             black = game.headers.get("Black", "Unknown")
#             result = game.headers.get("Result", "*")
            
#             # Write the initial position
#             writer.writerow([white, black, result, 0, "Initial", board.fen()])
            
#             # Process and write each move's FEN
#             for i, move in enumerate(moves, 1):
#                 board.push(move)
#                 writer.writerow([white, black, result, i, move.uci(), board.fen()])
        
#         print(f"Processed {game_count} games")

# # Usage
# pgn_file_path = '/kaggle/input/chess-pgn/combined_700-800.pgn'
# csv_file_path = '/kaggle/working/fen_data.csv'

# pgn_to_fen_csv(pgn_file_path, csv_file_path)


In [40]:
import dask.dataframe as dd

# Read the large CSV file using Dask
df = pd.read_csv('/kaggle/input/chess-data-subset/fen_subset.csv')

# Perform operations as you would with a Pandas DataFrame


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/chess-data-subset/fen_subset.csv'

In [ ]:
df['Move']=df['Move'].shift(-1)

In [ ]:
df

In [ ]:
df=df[df['Move']!= 'Initial']
df=df.dropna()
df.head()

In [41]:
sq_idx ={
     'a':0,
     'b':1,
     'c':2,
     'd':3,
     'e':4,
     'f':5,
     'g':6,
     'h':7
 }


def sq_to_idx(square):
  name=chess.square_name(square)
  return 8-int(name[1]),sq_idx[name[0]]


# Making a feature matrix from board.
Every 8x8 matrices out of first 12 matrices, store positions of different types of pieces, for both black and while the last two matrices stores squares available for a legal move, for both white and black respectively.


In [42]:
# def encode_board(fen):
#     board=chess.Board(fen)
#     board_features = np.zeros((14, 8, 8), dtype=int)

#     for piece in chess.PIECE_TYPES:
#         for square in board.pieces(piece, chess.WHITE):
#             idx = np.unravel_index(square, (8, 8))
#             board_features[piece-1][7-idx[0]][idx[1]] = 1

#         for square in board.pieces(piece, chess.BLACK):
#             idx = np.unravel_index(square, (8, 8))
#             board_features[piece-1][7-idx[0]][idx[1]] = 1

#     return board_features


In [43]:
from joblib import Parallel, delayed
import numpy as np
import chess

def encode_board(fen):
    board = chess.Board(fen)
    board_features = np.zeros((6, 8, 8), dtype=int)

    for piece in chess.PIECE_TYPES:
        # Encode white pieces
        for square in board.pieces(piece, chess.WHITE):
            row, col = 7 - square // 8, square % 8  # Convert square index to row, col
            board_features[piece-1, row, col] = 1

        # Encode black pieces
        for square in board.pieces(piece, chess.BLACK):
            row, col = 7 - square // 8, square % 8
            board_features[piece-1, row, col] = -1

    return board_features



In [44]:
encode_board('rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1')

array([[[ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0, -1,  0,  0,  0,  0, -1,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0, -1,  0,  0, -1,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  

In [45]:
# def encode_board_parallel(fen_array, n_jobs=-1):
#     # Use Parallel and delayed to distribute work across CPU cores
#     encoded_boards = Parallel(n_jobs=n_jobs)(
#         delayed(encode_board)(fen) for fen in fen_array
#     )
    
#     return np.array(encoded_boards)

# # Convert the DataFrame column to a NumPy array for faster processing
# fen_array = df['FEN'].to_numpy()

# # Process all FEN strings in parallel
# x_train = encode_board_parallel(fen_array)


# Sample board feature 3D matrix


#Encoding fen board and Algebraic moves.

Considering all possible information that a move can possess,we make a vector of 147 dimensions.

In [46]:
def move2mat(move,fen):
    board=chess.Board(fen)
    board.push_san(move).uci()
    move=str(board.pop())
    from_output_layer=np.zeros((8,8),dtype=int)
    from_row=8-int(move[1])
    from_col=sq_idx[move[0]]
    from_output_layer[from_row][from_col]=1


    to_output_layer=np.zeros((8,8),dtype=int)
    to_row=8-int(move[3])
    to_col=sq_idx[move[2]]
    to_output_layer[to_row][to_col]=1

    return np.stack([from_output_layer,to_output_layer])

In [47]:
move2mat('d7d5', 'rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1')

array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]])

In [48]:
# def encode_moves_batch(moves, initial_fen):
#     board = chess.Board(initial_fen)
#     encoded_moves = np.zeros((len(moves), 2, 8, 8), dtype=int)

#     for i, move in enumerate(moves):
#         encoded_moves[i] = move2mat(move, board)
    
#     return encoded_moves
# y_train = [encode_moves_batch(moves, initial_fen) for moves, initial_fen in zip(df['moves'], df['FEN'])]

In [49]:
class ChessDataset(Dataset):
    
    def __init__(self, df):
        """
        Args:
            df (pd.DataFrame): DataFrame with columns 'FEN' and 'Move'.
        """
        super(ChessDataset, self).__init__()
        self.df = df.reset_index(drop=True)  # Ensure proper indexing
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        # Randomly sample an index
        game_index = np.random.randint(0, len(self.df))
        
        # Retrieve FEN and move
        fen = self.df.loc[game_index, 'FEN']
        move = self.df.loc[game_index, 'Move']
        
    
        x = encode_board(fen)
        y = move2mat(move,fen)  # Assume move2mat encodes move
        
        # Apply board state adjustment if needed
        if game_index % 2 == 1:
            x *= -1
        
        return x, y


In [ ]:
x_train=ChessDataset(df[['FEN','Move']])
train_loader=DataLoader(x_train,batch_size=32,shuffle=True,drop_last=True)

In [50]:
class module(nn.Module):

    def __init__(self, hidden_size):
        super(module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_size)
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.activation1 = nn.SELU()
        self.activation2 = nn.SELU()

    def forward(self, x):
        x_input = torch.clone(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + x_input
        x = self.activation2(x)
        return x


In [51]:
class ChessNet(nn.Module):

    def __init__(self, hidden_layers=4, hidden_size=200):
        super(ChessNet, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, 3, stride=1, padding=1)
        self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)

        for i in range(self.hidden_layers):
            x = self.module_list[i](x)

        x = self.output_layer(x)
        return x


In [ ]:
# Initialize the model, loss function, and optimizer
model = ChessNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


In [ ]:
criterion_from = nn.CrossEntropyLoss()
criterion_to = nn.CrossEntropyLoss()



In [52]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
model = model.to(device)

In [53]:
device

device(type='cuda')

In [ ]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (x, y) in enumerate(train_loader):
        # Move inputs and labels to the GPU
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        # Forward pass
        output = model(x.float())  # Ensure x is of float type

        # Calculate losses
        loss_from = criterion_from(output[:, 0, :, :], y[:, 0, :, :].float())
        loss_to = criterion_to(output[:, 1, :, :], y[:, 1, :, :].float())
        loss = loss_from + loss_to

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 99:  # Print every 100 batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

    print(f'Epoch [{epoch + 1}/{num_epochs}] finished.')

print('Training finished.')

In [54]:
torch.device('cuda')

device(type='cuda')

In [55]:
model = ChessNet()
model=torch.load('/kaggle/input/model/pytorch/default/1/model2.pth')
model.to(device)

ChessNet(
  (input_layer): Conv2d(6, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (module_list): ModuleList(
    (0-3): 4 x module(
      (conv1): Conv2d(200, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(200, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): SELU()
      (activation2): SELU()
    )
  )
  (output_layer): Conv2d(200, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [56]:

def predict(model, x):
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation for inference
        # Move input to the GPU
        x = x.to(device).float()

        # Forward pass
        output = model(x)

        # Optionally, move the output back to the CPU
        output = output.cpu()

    return output


In [57]:
def check_mate_single(board):
    board = board.copy()
    legal_moves = list(board.legal_moves)
    for move in legal_moves:
        board.push_uci(str(move))
        if board.is_checkmate():
            move = board.pop()
            return move
        _ = board.pop()

In [58]:
def distribution_over_moves(vals):
    probs = np.array(vals)
    probs = np.exp(probs)
    probs = probs / probs.sum()
    probs = probs ** 3
    probs = probs / probs.sum()
    return probs

In [59]:
def choose_move(fen, color):
    board=chess.Board()
    legal_moves = list(board.legal_moves)
    
    move = check_mate_single(board)
    if move is not None:
        return move
    
    x = torch.Tensor(encode_board(fen)).float().to('cuda')
    if color == chess.BLACK:
        x *= -1
    x = x.unsqueeze(0)
    move = predict(model,x)
    move=torch.reshape(move,(2,8,8))
    
    froms = [str(legal_move)[:2] for legal_move in legal_moves]
    froms = list(set(froms))
    vals=[]

    for from_ in froms:
        val = move[0,:,:][8- int(from_[1]),sq_idx[from_[0]]] 
        vals.append(val)
    probs = distribution_over_moves(vals)
    
    choosen_from = str(np.random.choice(froms, size=1, p=probs)[0])[:2]
    vals = []

    for legal_move in legal_moves:
        # Extract the 'from' and 'to' parts of the move
        from_ = str(legal_move)[:2]
        to = str(legal_move)[2:]

        # Check if the move's 'from' part matches the chosen 'from' square
        if from_ == choosen_from:
            val_move = move[1,:,:][8- int(to[1]),sq_idx[to[0]]]          # Replace with the actual value calculation
            vals.append(val_move)
        else:
            vals.append(0)

    max_val_index = np.argmax(vals)
    choosen_move = legal_moves[max_val_index]

    return choosen_move    

In [60]:
choose_move('rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1','BLACK')

Move.from_uci('g1h3')

#Creating a decode function for our output logits.
Similar to our encoding function we create decoding function to get back to the algebric move from our encoded vector.